In [1]:
import numpy as np
from scipy.special import logsumexp
from numpy.linalg import matrix_power
from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.stats import multivariate_normal
from scipy.stats import norm
import matplotlib.pyplot as plt
from numpy.polynomial.hermite import hermgauss

# PS0 Empirical IO: Numerical Stuff

## Part 0: Logit Inclusive Value

### 1 - Convexity

The logit inclusive value is the log of the denominator of a logit probability of choice. It is as follows:

$$
IV = \log \sum_{i=0}^N \exp(x_i)
$$

Suppose $x_0 = 0$. Then, $IV(x_1, ... , x_n) = \log \sum_{i=1}^N (1+\exp(x_i))$. We want to show that this function is convex everywhere. The second derivative of this function is:
$$
\frac{\partial^2}{\partial x_i^2} \log\left(N + \sum_{j=1}^N \exp(x_j)\right) = \exp(2x_i) \left( \frac{S - 1}{S^2} \right)
 \\
S = N + \sum_{j=1}^N \exp(x_j).
$$

Which is indeed positive. Therefore, this function is convex.

### 2- Large $x_i$ problem

For large values of $x_i$, we cannot manage to find the exponential.

To resolve this issue, we now that, given $m = \max_i x_i$:

$$
IV = \log \sum_{i=0}^N \exp(x_i) = \log \left(\exp(m)\sum_{i=0}^N \exp(x_i - m)\right)
$$
Therefore, we have:

$$
IV = m + \log \sum_{i=0}^N \exp(x_i - m)
$$

In [7]:
def solve_IV(x):
    m = np.max(x)
    IV = m + np.log(np.sum(np.exp(x - m)))
    return IV

In [22]:
x = np.array([300, 400, 500, 750, 799, 800])
IV_stable = solve_IV(x)
print("Using the above method, IV is:", IV_stable)
IV_unstable = np.log(np.sum(np.exp(x)))
print("Using the naive method, IV is:", IV_unstable)

Using the above method, IV is: 800.3132616875182
Using the naive method, IV is: inf


C:\Users\moham\AppData\Local\Temp\ipykernel_24228\1108397953.py:4: RuntimeWarning: overflow encountered in exp
  IV_unstable = np.log(np.sum(np.exp(x)))


### 3. logsumexp function in python

In [24]:
IV_stable_python = logsumexp(x)
print("Using scipy's logsumexp, IV is:", IV_stable_python)

Using scipy's logsumexp, IV is: 800.3132616875182


This function in python uses the same method as explained above.

## Part 1: Markov Chains

Here, the goal is to write a function to solve for the ergodic distribution of a markov process, given its transition matrix. 

To do this properly, we can use the idea of eigenvalues and eigen vectors. We know that, the ergodic distribution is the fixed point of $\pi P$. Therefore:

$$
\pi = \pi P \rightarrow (P' - I)\pi' = 0
$$

So, the ergodic distribution is the normalized eigenvector (should be added to one) corresponding to eigenvalue of 1 for matrix $P'$.

In [43]:
import numpy as np

def ergodic_distribution(P):
    # Step 1: Compute the eigenvalues and eigenvectors of the transpose of P
    eigenvalues, eigenvectors = np.linalg.eig(P.T)
    
    # Step 2: Find the eigenvector corresponding to the eigenvalue 1
    # We assume there is exactly one eigenvalue that is 1 for a valid transition matrix
    eigenvector = eigenvectors[:, np.isclose(eigenvalues, 1)]
    
    # Step 3: Normalize the eigenvector
    steady_state = eigenvector[:, 0]  # Extract the eigenvector (as it's returned in a 2D array)
    steady_state = steady_state / np.sum(steady_state)
    
    # Ensure the result is real (numerical computations may result in a complex vector with very small imaginary parts)
    steady_state = steady_state.real
    
    return steady_state

In [44]:
P = np.array([[0.2, 0.4, 0.4],
              [0.1, 0.3, 0.6],
              [0.5, 0.1, 0.4]])
steady_state = ergodic_distribution(P)
print("The steady state distribution is:", steady_state.round(3))

The steady state distribution is: [0.31  0.241 0.448]


Now, notice that:
$$
\pi_t = \pi_0 P^t
$$
So, to find the ergodic distribution, we can set $\pi_0 = [1, 0, ..., 0]$ and find $\pi_t$ for large values of $t$.

In [49]:
pi_0 = np.array([1, 0, 0])
ergodic = pi_0 @ matrix_power(P, 100)
print("The ergodic distribution is:", ergodic.round(3))

The ergodic distribution is: [0.31  0.241 0.448]


One can see that these are the same results. When reached to the ergodic distribution, the distribution of transitioning from each alternative is the same and equal to the ergodic distribution.

## Part 2: Numerical Integration

Here, the goal is to solve for the logit choice probability by numerical integration. 

$$
p(X, \theta)=\int_{-\infty}^{\infty} \frac{\exp \left(\beta_i X\right)}{1+\exp \left(\beta_i X\right)} f\left(\beta_i \mid \theta\right) \partial \beta_i \,\, ,\,\, f\sim N(0.5,2) \,\, ,\,\, X = 0.5
$$

### True Value

Using the quad function in python, we have:

In [185]:
X = 0.5
mean = 0.5
sigma = np.sqrt(2)
integral_true = quad(lambda beta: np.exp(beta*X)/(1 + np.exp(beta*X))
                * norm.pdf(beta, mean, sigma), -20, 20, epsabs=1e-14)[0]
print("The integral is:", integral_true)

The integral is: 0.555939162843465


### Approximation using Monte Carlo method

The method is as follows (for 1D integral):

$$
I = \int_a^b f(x) \, dx \rightarrow \hat{I} = (b-a) \frac{1}{N} \sum_{i=1}^N f(x_i)
$$
where, $x_i$ s are uniformly randomly drawn in $(a,b)$.

In [167]:
def binomiallogit(beta, params):
    X = params[0]
    mean = params[1]
    sigma = params[2]
    return np.exp(beta*X)/(1 + np.exp(beta*X)) * norm.pdf(beta, mean, sigma)
    
def monte_carlo_integration(func, up_lim, low_lim,
                            params, n_samples=1000):
    samples = np.random.uniform(low_lim, up_lim, size=n_samples)
    # samples = np.linspace(low_lim, up_lim, n_samples)
    integral = (up_lim - low_lim) * np.mean(func(samples, params))
    return integral

In [178]:
params = [X, mean, sigma]
iter = 10
integral_MC20 = 0
integral_MC400 = 0
for i in range(iter):
    integral_MC20 += monte_carlo_integration(binomiallogit, 10, -10, params, n_samples=20)
integral_MC20 /= iter
for i in range(iter):
    integral_MC400 += monte_carlo_integration(binomiallogit, 10, -10, params, n_samples=400)
integral_MC400 /= iter

print("The integral using 20 samples is:", integral_MC20)
print("The integral using 400 samples is:", integral_MC400)

The integral using 20 samples is: 0.6915976993037517
The integral using 400 samples is: 0.5457227383948215


### Approximation using Gauss-Hermite Quadrature

GM is used for when the goal is to solve for some integral of the following form, with the following approximation and weights.

$$
\int_{-\infty}^{\infty} e^{-x^2} f(x) \, dx \approx \sum_{i=1}^{n} w_i f(x_i)
\,\, ,\,\, w_i = \frac{2^{n-1} n! \sqrt{\pi}}{n^2 \left[ H_{n-1}(x_i) \right]^2}
$$

So, as in our case, the integral is as follows:

$$
I = \int_{-\infty}^{\infty} \frac{\exp \left(\beta_i X\right)}{1+\exp \left(\beta_i X\right)} 
\frac{1}{\sqrt{2\pi} \sigma} \exp\left(-\left(\frac{\beta_i - \mu}{\sqrt{2}\sigma}\right)^2\right)\partial \beta_i
$$
Then, define $y = \frac{\beta_i - \mu}{\sqrt{2} \sigma}$, then:

$$
I = \frac{1}{\sqrt{\pi}} \int_{-\infty}^\infty \frac{\exp \left((\sqrt{2} \sigma y + \mu) X\right)}{1+\exp \left((\sqrt{2} \sigma y + \mu) X\right)} e^{-y^2} \, dy
$$
So, using the GH formula, we have (define $g(y) = \frac{\exp \left((\sqrt{2} \sigma y + \mu) X\right)}{1+\exp \left((\sqrt{2} \sigma y + \mu) X\right)} $)
$$
I \approx \frac{1}{\sqrt{\pi}} \sum_{i=1}^{n} w_i g(x_i)
$$

In [173]:
def G_func(roots, params):
    X = params[0]
    mean = params[1]
    sigma = params[2]
    beta = np.sqrt(2) * sigma * roots + mean
    g = np.exp(beta*X)/(1 + np.exp(beta*X))
    return g
def Gauss_Hermite_integral(n, params):
    roots, weights = hermgauss(n)
    g = G_func(roots, params)
    return np.sum(weights * g) / np.sqrt(np.pi)

In [174]:
integral_GH = Gauss_Hermite_integral(10, params)
print("The integral using Gauss-Hermite quadrature is:", integral_GH)

The integral using Gauss-Hermite quadrature is: 0.5559391582008274


So, for summary, here are the results for different methods:

In [179]:
print("True Value:", integral_true)
print("Approximation using Monte Carlo method (20 samples):", integral_MC20)
print("Approximation using Monte Carlo method (400 samples):", integral_MC400)
print("Approximation using Gauss-Hermite Quadrature:", integral_GH)

True Value: 0.555939162843465
Approximation using Monte Carlo method (20 samples): 0.6915976993037517
Approximation using Monte Carlo method (400 samples): 0.5457227383948215
Approximation using Gauss-Hermite Quadrature: 0.5559391582008274


### Two dimension case

Now, suppose that:

$$
f \sim N\left((0.5,1) , \begin{pmatrix}
4 & 0 \\ 0 & 1
\end{pmatrix}\right) \,\, ,\,\, X = (0.5, 1)
$$

In [3]:
def multivariate_normal_pdf(x, mean, cov):
    result = np.zeros((x.shape[0]))
    for i in range(x.shape[0]):
        result[i] = multivariate_normal.pdf(x[i], mean=mean, cov=cov)
    return result

#### True Value

In [4]:
X = np.array([0.5,1])
mu = np.array([0.5,1])
var = np.array([[4,0],[0,1]])

integral_true_2D = dblquad(lambda beta_1, beta_2: 
    np.exp(np.dot(np.array([beta_1,beta_2]),X))/(1 + np.exp(
        np.dot(np.array([beta_1,beta_2]),X)))* multivariate_normal_pdf(
            np.array([[beta_1,beta_2]]), mean=mu, cov=var), 
        -20, 20, -20, 20, epsabs=1e-14)[0]
print("The true integral is:", integral_true_2D)

The true integral is: 0.7144838053940904


#### Approximation using Monte Carlo method

Same as above. Though, we should multiply the average by the volume of the integraion space. In other words:

$$
I = \int_{\mathbb{V}} f(\boldsymbol{x}) \,d\boldsymbol{x} \rightarrow \hat{I}_{MC_N} = \text{Volume}(\mathbb{V}) \times \frac{1}{N} \sum_{i=1}^N f(\boldsymbol{x}_i)

$$

In [47]:
def binomiallogit2D(beta_1, beta_2, params):
    X = params[0]
    mean = params[1]
    sigma = params[2]
    return np.exp(np.array([beta_1, beta_2]).T @ X)/(1 +
                                   np.exp(np.array([beta_1,beta_2]).T @ X)
                                   ) * multivariate_normal_pdf(
                                       np.array([beta_1,beta_2]).T, mean, sigma
                                       )
    
def monte_carlo_integration2D(func, low_lim, up_lim,
                            params, n_samples=1000):
    beta_1 = np.random.uniform(low_lim, up_lim, size=n_samples)
    beta_2 = np.random.uniform(low_lim, up_lim, size=n_samples)
    integral = (up_lim - low_lim)**2 * np.mean(func(beta_1, beta_2, params))
    return integral

In [54]:
iter = 10
integral_MC202D = 0
integral_MC4002D = 0
for i in range(iter):
    integral_MC202D += monte_carlo_integration2D(binomiallogit2D, -10, 10, [X, mu, var], n_samples=20)
integral_MC202D /= iter
for i in range(iter):
    integral_MC4002D += monte_carlo_integration2D(binomiallogit2D, -10, 10, [X, mu, var], n_samples=400)
integral_MC4002D /= iter

print("The integral using 20 samples is:", integral_MC202D)
print("The integral using 400 samples is:", integral_MC4002D)

The integral using 20 samples is: 0.6318587213389532
The integral using 400 samples is: 0.6959738211991421


### Approximation using Gauss-Hermite Quadrature

For the 2D case, we can use the following idea:

$$
\int_{-\infty}^{\infty} \int_{-\infty}^{\infty} e^{-(x^2 + y^2)} f(x, y) \, dx \, dy \approx \sum_{i=1}^n \sum_{j=1}^n w_i w_j f(x_i, y_j)
$$

For our case, the integral is as follows:

$$
I = \frac{1}{2\pi\sigma_1\sigma_2}\int_{\beta_1}\int_{\beta_2} 
\frac{\exp(\beta'X)}{1+\exp{\beta'X}} \exp{\left\{-\left(\left(\frac{\beta_1 - \mu_1}{\sqrt{2}\sigma_1}\right)^2 + \left(\frac{\beta_2 - \mu_2}{\sqrt{2}\sigma_2}\right)^2\right)\right\}}\,d\beta_1\,d\beta_2
$$

So, defining, $y_1 = \frac{\beta_1 - \mu_1}{\sqrt{2}\sigma_1} , y_2 = \frac{\beta_2 - \mu_2}{\sqrt{2}\sigma_2}$, we have:
$$
I = \frac{1}{\pi} \int\int \frac{\exp((\sqrt{2}\sigma_1 y_1 + \mu_1) X_1 + (\sqrt{2}\sigma_2 y_2 + \mu_2) X_2)}{1+\exp{(\sqrt{2}\sigma_1 y_1 + \mu_1) X_1 + (\sqrt{2}\sigma_2 y_2 + \mu_2) X_2)}} e^{-(y_1^2 + y_2^2)} \,dy_1 \, dy_2
$$

Therefore:

$$
\hat{I}_{GH_N} = \frac{1}{\pi} \sum_{i=1}^n \sum_{j=1}^n w_i w_j g(y_{1,i}, y_{2,j})

$$

In [128]:
def G_func_2D(roots, params, n):
    X = params[0]
    mean = params[1]
    sigma = params[2]
    beta = roots * (np.sqrt(2) * sigma) + mean
    g = np.exp(beta @ X)/(1 + np.exp(beta @ X))
    g = g.reshape([n, n])
    return g

def Gauss_Hermite_integral_2D(n, params):
    roots, weights = hermgauss(n)
    Y_1, Y_2 = np.meshgrid(roots, roots)
    W_1, W_2 = np.meshgrid(weights, weights)
    roots = np.array([Y_1.flatten(), Y_2.flatten()]).T
    g = G_func_2D(roots, params, n)
    result = np.sum(W_1 * W_2 * g) / np.pi
    return result

In [138]:
sigma = np.array([np.sqrt(var[0,0]), np.sqrt(var[1,1])])
integral_GH_2D = Gauss_Hermite_integral_2D(10, [X, mu, sigma])
print("The integral using Gauss-Hermite quadrature is:", integral_GH_2D)

The integral using Gauss-Hermite quadrature is: 0.7144838072089089


In [139]:
print("True Value:", integral_true_2D)
print("Approximation using Monte Carlo method (20 samples):", integral_MC202D)
print("Approximation using Monte Carlo method (400 samples):", integral_MC4002D)
print("Approximation using Gauss-Hermite Quadrature:", integral_GH_2D)

True Value: 0.7144838053940904
Approximation using Monte Carlo method (20 samples): 0.6318587213389532
Approximation using Monte Carlo method (400 samples): 0.6959738211991421
Approximation using Gauss-Hermite Quadrature: 0.7144838072089089


One can infer how Gauss-Hermite is the most efficient way to do this integrals numerically.